# Imports

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
import json

In [4]:

spark = SparkSession.builder.getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/10 14:42:17 WARN Utils: Your hostname, HMLaptop, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/10 14:42:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 14:42:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.option("multiline", "true").json("../data/all_dns.json")
    

In [7]:
df.show(2)

+-------+-------------+--------+-----------+-----+----------------+-----------+------+-----------+---+---------+-----+-------------+--------+-----------+--------------------+-----+
|answers|       dst_ip|dst_port|dst_service|   id|ip_header_length|ip_protocol|opcode|packet_size| qr|questions|rcode|       src_ip|src_port|src_service|           timestamp| type|
+-------+-------------+--------+-----------+-----+----------------+-----------+------+-----------+---+---------+-----+-------------+--------+-----------+--------------------+-----+
|     []|192.168.20.72|      53|     domain|63107|              20|        UDP|     0|        123|  0|       []|    0|192.168.20.38|   52763|       NULL|2020-11-24T00:28:...|other|
|     []|192.168.20.38|   52763|       NULL|63107|              20|        UDP|     0|        138|  1|       []|    0|192.168.20.72|      53|     domain|2020-11-24T00:28:...|other|
+-------+-------------+--------+-----------+-----+----------------+-----------+------+---------

In [100]:
dns_data = df.select("timestamp","src_ip", "dst_ip","id", "opcode","qr","rcode","questions","answers")

In [139]:
all_queries = dns_data.filter((dns_data["opcode"] == 0) & (dns_data["qr"] == 0))
all_responses = dns_data.filter((dns_data["opcode"] == 0) & (dns_data["qr"] == 1))
valid_queries = all_queries.filter(sf.size("questions") > 0).drop("answers")
valid_responses = all_responses.filter(sf.size("answers") > 0).drop("questions")
empty_questions = all_queries.filter(sf.size("questions") == 0)
empty_answers = all_responses.filter(sf.size("answers") == 0)

In [140]:
valid_queries.show(3)

+--------------------+-------------+-------+-----+------+---+-----+--------------------+
|           timestamp|       src_ip| dst_ip|   id|opcode| qr|rcode|           questions|
+--------------------+-------------+-------+-----+------+---+-----+--------------------+
|2020-11-24T00:32:...|192.168.20.38|8.8.8.8|53331|     0|  0|    0|[{1, v10.events.d...|
|2020-11-24T00:37:...|192.168.20.38|8.8.8.8|11770|     0|  0|    0|[{1, dns.google.,...|
|2020-11-24T00:42:...|192.168.20.38|8.8.8.8|64588|     0|  0|    0|[{1, tile-service...|
+--------------------+-------------+-------+-----+------+---+-----+--------------------+
only showing top 3 rows


In [141]:
valid_responses.show(3)

+--------------------+-------+-------------+-----+------+---+-----+--------------------+
|           timestamp| src_ip|       dst_ip|   id|opcode| qr|rcode|             answers|
+--------------------+-------+-------------+-----+------+---+-----+--------------------+
|2020-11-24T00:37:...|8.8.8.8|192.168.20.38|11770|     0|  1|    0|[{1, 8.8.8.8, dns...|
|2020-11-24T00:42:...|8.8.8.8|192.168.20.38|19842|     0|  1|    0|[{1, 8.8.4.4, dns...|
|2020-11-24T00:54:...|8.8.8.8|192.168.20.38|17689|     0|  1|    0|[{1, 8.8.4.4, dns...|
+--------------------+-------+-------------+-----+------+---+-----+--------------------+
only showing top 3 rows


We only need queries with actual questions

In [142]:
valid_queries.show(3)


+--------------------+-------------+-------+-----+------+---+-----+--------------------+
|           timestamp|       src_ip| dst_ip|   id|opcode| qr|rcode|           questions|
+--------------------+-------------+-------+-----+------+---+-----+--------------------+
|2020-11-24T00:32:...|192.168.20.38|8.8.8.8|53331|     0|  0|    0|[{1, v10.events.d...|
|2020-11-24T00:37:...|192.168.20.38|8.8.8.8|11770|     0|  0|    0|[{1, dns.google.,...|
|2020-11-24T00:42:...|192.168.20.38|8.8.8.8|64588|     0|  0|    0|[{1, tile-service...|
+--------------------+-------------+-------+-----+------+---+-----+--------------------+
only showing top 3 rows


In [143]:
valid_queries = valid_queries.withColumn("questions", sf.explode("questions"))
valid_queries = valid_queries.withColumns({
    "qname": valid_queries.questions.qname,
    "qtype": valid_queries.questions.qtype,
    "qlen": sf.length(valid_queries.questions.qname)}).drop("questions")

In [144]:
valid_queries.show(3)

+--------------------+-------------+-------+-----+------+---+-----+--------------------+-----+----+
|           timestamp|       src_ip| dst_ip|   id|opcode| qr|rcode|               qname|qtype|qlen|
+--------------------+-------------+-------+-----+------+---+-----+--------------------+-----+----+
|2020-11-24T00:32:...|192.168.20.38|8.8.8.8|53331|     0|  0|    0|v10.events.data.m...|    1|  30|
|2020-11-24T00:37:...|192.168.20.38|8.8.8.8|11770|     0|  0|    0|         dns.google.|    1|  11|
|2020-11-24T00:42:...|192.168.20.38|8.8.8.8|64588|     0|  0|    0|tile-service.weat...|    1|  35|
+--------------------+-------------+-------+-----+------+---+-----+--------------------+-----+----+
only showing top 3 rows


In [145]:
valid_responses = valid_responses.withColumn("answers", sf.explode("answers")).drop("questions")
valid_responses = valid_responses.withColumns({
    "rclass":valid_responses.answers.rclass,
    "rdata": valid_responses.answers.rdata,
    "rrname": valid_responses.answers.rrname,
    "rtype": valid_responses.answers.rtype,
    "ttl": valid_responses.answers.ttl
}).drop("answers", "opcode")


In [146]:
valid_responses.show(3)

+--------------------+-------+-------------+-----+---+-----+------+-------+-----------+-----+---+
|           timestamp| src_ip|       dst_ip|   id| qr|rcode|rclass|  rdata|     rrname|rtype|ttl|
+--------------------+-------+-------------+-----+---+-----+------+-------+-----------+-----+---+
|2020-11-24T00:37:...|8.8.8.8|192.168.20.38|11770|  1|    0|     1|8.8.8.8|dns.google.|    1|156|
|2020-11-24T00:42:...|8.8.8.8|192.168.20.38|19842|  1|    0|     1|8.8.4.4|dns.google.|    1|631|
|2020-11-24T00:54:...|8.8.8.8|192.168.20.38|17689|  1|    0|     1|8.8.4.4|dns.google.|    1| 84|
+--------------------+-------+-------------+-----+---+-----+------+-------+-----------+-----+---+
only showing top 3 rows


In [147]:
valid_responses = valid_responses.withColumnsRenamed({
    "timestamp": "ts",
    "src_ip": "ns",
    "dst_ip": "client_ip",
    "id": "rid",
    "qr": "rqr",
    "rcode": "rrcode"
})

In [148]:
valid_responses.show(3)

+--------------------+-------+-------------+-----+---+------+------+-------+-----------+-----+---+
|                  ts|     ns|    client_ip|  rid|rqr|rrcode|rclass|  rdata|     rrname|rtype|ttl|
+--------------------+-------+-------------+-----+---+------+------+-------+-----------+-----+---+
|2020-11-24T00:37:...|8.8.8.8|192.168.20.38|11770|  1|     0|     1|8.8.8.8|dns.google.|    1|156|
|2020-11-24T00:42:...|8.8.8.8|192.168.20.38|19842|  1|     0|     1|8.8.4.4|dns.google.|    1|631|
|2020-11-24T00:54:...|8.8.8.8|192.168.20.38|17689|  1|     0|     1|8.8.4.4|dns.google.|    1| 84|
+--------------------+-------+-------------+-----+---+------+------+-------+-----------+-----+---+
only showing top 3 rows
